In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import zipfile

z= zipfile.ZipFile('../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')
z.extractall()

In [4]:
train = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')

In [5]:
cols_target = ['obscene','insult','toxic','severe_toxic','identity_hate','threat']

In [6]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
train.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [8]:
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
#we dont have to use id here
train.drop(['id'],axis=1,inplace=True)

In [10]:
train.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [11]:
train.shape

(159571, 7)

In [12]:
import re

In [13]:
train["comment_text"]=train["comment_text"].apply(lambda x: x.lower())

In [14]:
train["comment_text"]=train["comment_text"].apply(lambda x: re.sub('\w*\d\w*','', x))

In [15]:
test.shape

(153164, 2)

In [16]:
import string
train["comment_text"]=train["comment_text"].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))

In [17]:
def clean_text(text):
    text = re.sub(r"\n", " ", text)
    return text
train["comment_text"] = train["comment_text"].apply(clean_text)
train["comment_text"]

0         explanation why the edits made under my userna...
1         daww he matches this background colour im seem...
2         hey man im really not trying to edit war its j...
3          more i cant make any real suggestions on impr...
4         you sir are my hero any chance you remember wh...
                                ...                        
159566    and for the second time of asking when your vi...
159567    you should be ashamed of yourself   that is a ...
159568    spitzer   umm theres no actual article for pro...
159569    and it looks like it was actually you who put ...
159570     and  i really dont think you understand  i ca...
Name: comment_text, Length: 159571, dtype: object

In [18]:
train

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,explanation why the edits made under my userna...,0,0,0,0,0,0
1,daww he matches this background colour im seem...,0,0,0,0,0,0
2,hey man im really not trying to edit war its j...,0,0,0,0,0,0
3,more i cant make any real suggestions on impr...,0,0,0,0,0,0
4,you sir are my hero any chance you remember wh...,0,0,0,0,0,0
...,...,...,...,...,...,...,...
159566,and for the second time of asking when your vi...,0,0,0,0,0,0
159567,you should be ashamed of yourself that is a ...,0,0,0,0,0,0
159568,spitzer umm theres no actual article for pro...,0,0,0,0,0,0
159569,and it looks like it was actually you who put ...,0,0,0,0,0,0


In [19]:
train.to_csv('train.csv',index=False)

In [20]:
train.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,explanation why the edits made under my userna...,0,0,0,0,0,0
1,daww he matches this background colour im seem...,0,0,0,0,0,0
2,hey man im really not trying to edit war its j...,0,0,0,0,0,0
3,more i cant make any real suggestions on impr...,0,0,0,0,0,0
4,you sir are my hero any chance you remember wh...,0,0,0,0,0,0


In [21]:
X = train.comment_text

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=5000,stop_words='english')
vect

TfidfVectorizer(max_features=5000, stop_words='english')

In [23]:
type(X)

pandas.core.series.Series

In [24]:
test.shape

(153164, 2)

In [25]:
test_df = test.comment_text

In [26]:
X_train = vect.fit_transform(X.astype('U'))
X_test = vect.transform(test_df.astype('U'))

In [27]:
X_test

<153164x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 2582096 stored elements in Compressed Sparse Row format>

In [28]:
X_train

<159571x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 3074764 stored elements in Compressed Sparse Row format>

In [29]:
import sklearn

In [30]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(C=24.0)

In [31]:
submission_binary1 = pd.read_csv('./sample_submission.csv')
submission_binary1.shape

(153164, 7)

In [32]:
from sklearn.metrics import classification_report
for label in cols_target:
    print('... Processing {}'.format(label))
    y = train[label]
    # train the model using X_dtm & y
    logreg.fit(X_train, y)
    # compute the training accuracy
    y_pred_X = logreg.predict(X_train)
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
    print(classification_report(y,y_pred_X))
    # compute the predicted probabilities for X_test_dtm
    test_y_prob = logreg.predict_proba(X_test)[:,1]
    print(test_y_prob.shape)
    submission_binary1[label] = test_y_prob

... Processing obscene


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Training accuracy is 0.9835621760846269
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    151122
           1       0.91      0.77      0.83      8449

    accuracy                           0.98    159571
   macro avg       0.95      0.88      0.91    159571
weighted avg       0.98      0.98      0.98    159571

(153164,)
... Processing insult


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Training accuracy is 0.9760232122378126
              precision    recall  f1-score   support

           0       0.98      0.99      0.99    151694
           1       0.83      0.65      0.73      7877

    accuracy                           0.98    159571
   macro avg       0.91      0.82      0.86    159571
weighted avg       0.97      0.98      0.97    159571

(153164,)
... Processing toxic


...

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Training accuracy is 0.9923795677159384
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    157976
           1       0.71      0.41      0.52      1595

    accuracy                           0.99    159571
   macro avg       0.85      0.70      0.76    159571
weighted avg       0.99      0.99      0.99    159571

(153164,)
... Processing identity_hate


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Training accuracy is 0.9944037450413923
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    158166
           1       0.82      0.46      0.59      1405

    accuracy                           0.99    159571
   macro avg       0.91      0.73      0.80    159571
weighted avg       0.99      0.99      0.99    159571

(153164,)
... Processing threat
Training accuracy is 0.9985085009180866
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    159093
           1       0.90      0.56      0.69       478

    accuracy                           1.00    159571
   macro avg       0.95      0.78      0.85    159571
weighted avg       1.00      1.00      1.00    159571

(153164,)


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [33]:
train 

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,explanation why the edits made under my userna...,0,0,0,0,0,0
1,daww he matches this background colour im seem...,0,0,0,0,0,0
2,hey man im really not trying to edit war its j...,0,0,0,0,0,0
3,more i cant make any real suggestions on impr...,0,0,0,0,0,0
4,you sir are my hero any chance you remember wh...,0,0,0,0,0,0
...,...,...,...,...,...,...,...
159566,and for the second time of asking when your vi...,0,0,0,0,0,0
159567,you should be ashamed of yourself that is a ...,0,0,0,0,0,0
159568,spitzer umm theres no actual article for pro...,0,0,0,0,0,0
159569,and it looks like it was actually you who put ...,0,0,0,0,0,0


In [34]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['none'] = 1-train[label_cols].max(axis=1)

In [35]:
train

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
0,explanation why the edits made under my userna...,0,0,0,0,0,0,1
1,daww he matches this background colour im seem...,0,0,0,0,0,0,1
2,hey man im really not trying to edit war its j...,0,0,0,0,0,0,1
3,more i cant make any real suggestions on impr...,0,0,0,0,0,0,1
4,you sir are my hero any chance you remember wh...,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
159566,and for the second time of asking when your vi...,0,0,0,0,0,0,1
159567,you should be ashamed of yourself that is a ...,0,0,0,0,0,0,1
159568,spitzer umm theres no actual article for pro...,0,0,0,0,0,0,1
159569,and it looks like it was actually you who put ...,0,0,0,0,0,0,1


In [36]:
max_features = 20000
maxlen = 200

In [37]:
list_sentences_train = train["comment_text"].values
list_sentences_test = test["comment_text"].values

In [38]:
y = train[label_cols].values

In [39]:
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint


In [40]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)


In [41]:
output_bias = None

In [42]:

def get_model(output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Bidirectional(LSTM(50, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid",bias_initializer=output_bias)(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model


model = get_model()
print(model.summary())
batch_size = 32
epochs = 2

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 128)          2560000   
_________________________________________________________________
bidirectional (Bidirectional (None, 200, 100)          71600     
_________________________________________________________________
global_max_pooling1d (Global (None, 100)               0         
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0     

In [43]:


file_path="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor="val_loss", mode="min", patience=20)


callbacks_list = [checkpoint, early] #early
model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)

model.load_weights(file_path)

y_test = model.predict(X_te)



sample_submission = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip")

sample_submission[label_cols] = y_test



sample_submission.to_csv("baseline.csv", index=False)

Epoch 1/2
4488/4488 [==============================] - 208s 45ms/step - loss: 0.0898 - accuracy: 0.8818 - val_loss: 0.0504 - val_accuracy: 0.9939

Epoch 00001: val_loss improved from inf to 0.05043, saving model to weights_base.best.hdf5
Epoch 2/2
4488/4488 [==============================] - 199s 44ms/step - loss: 0.0453 - accuracy: 0.9885 - val_loss: 0.0487 - val_accuracy: 0.9937

Epoch 00002: val_loss improved from 0.05043 to 0.04871, saving model to weights_base.best.hdf5
